In [97]:
### import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Feature Selection

In [98]:
# Correlation
def load_data_corr():
    data = pd.read_csv('heart.csv')
    #Find correlation between features
    corr = data.corr()
    sns.heatmap(corr,annot=True, linewidths=1.5,fmt=".1f")

    plt.show()
    corr_target = abs(corr["target"])
    relevant_feature = corr_target[corr_target > 0.3]
    print(relevant_feature)
    y = data["target"]
    X = data[["cp","thalach","exang","oldpeak","slope","ca","thal"]]
    print(X.shape)
    return X,y

In [99]:
#PCA
def load_data_pca(normalized):
    pca = PCA()
    pca.fit(normalized)
    plt.plot(pca.explained_variance_ratio_.cumsum())
    plt.show()
    pca= PCA(n_components=2)
    component = pca.fit_transform(normalized)
    return component

In [100]:
#Load data
def load_data():
    data = pd.read_csv('heart.csv')
    y = data["target"]
    X = data.drop('target',axis=1)
    data.hist(figsize=(20,30))
    return X,y

In [101]:
#Feature importance
def load_data_f_importance():
    data = pd.read_csv('heart.csv')
    y = data["target"]
    X = data[["cp","thalach","oldpeak","sex","ca","thal"]]
    return X,y

In [102]:
# inputs,target = load_data_corr()
# inputs,target = load_data()#Without feature selection
# inputs,target = load_data_f_importance()
from sklearn.preprocessing import MinMaxScaler
mins  = MinMaxScaler(inputs)
inputs = load_data_pca(inputs)

In [103]:
# Splitting data
X_train, X_test, y_train, y_test = train_test_split(inputs, target, test_size=0.20, random_state = 0)

In [104]:
#PREPROCESSING
from sklearn.preprocessing import MinMaxScaler
def pre_process(X_train,X_test):
    scaler = StandardScaler()
    X_train = MinMaxScaler().fit_transform(X_train)
    X_test = MinMaxScaler().fit_transform(X_test)
    # X_train = scaler.fit_transform(X_train)
    # X_test = scaler.fit_transform(X_test)
    return X_train,X_test
# X_train,X_test = pre_process(X_train,X_test)

# SVM

In [105]:
#Create model
svc =  SVC(kernel='rbf' ,C=0.5)
svc.fit(X_train, y_train)
#Predict model
svc_predicted = svc.predict(X_test)

# Evaluation

In [106]:
svc_conf_matrix = confusion_matrix(y_test, svc_predicted)
svc_acc_score = accuracy_score(y_test, svc_predicted)
print("confussion matrix")
print(svc_conf_matrix)
print("Accuracy of Support Vector Classifier with feature selection:",svc_acc_score*100,'\n')

confussion matrix
[[14 13]
 [ 4 30]]
Accuracy of Support Vector Classifier with feature selection: 72.1311475409836 



In [107]:
# Cross validation
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
mins  = MinMaxScaler(inputs)
score = cross_val_score(svc, inputs, target,cv=5)
print(f"Cross Validation Accuracy: {score.mean()*100}")

Cross Validation Accuracy: 68.3224043715847


In [108]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
#Choose 10 neighbors beside of 11.. 
knn = KNeighborsClassifier(n_neighbors=10)
mins  = MinMaxScaler(inputs)
# inputs = scaler.fit_transform(inputs)
score = cross_val_score(knn, inputs, target,cv=5)
print(f"Cross Validation Accuracy: {score.mean()*100}")

Cross Validation Accuracy: 72.60109289617486


In [109]:
#XGB
from xgboost import XGBClassifier
xgb = XGBClassifier(learning_rate=0.01,n_estimators=25 , booster='dart')
xgb.fit(X_train, y_train)
xgb_predicted = xgb.predict(X_test)
acc = accuracy_score(y_test, xgb_predicted)
print("Accuracy XGB: ",acc)

[12:28:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy XGB:  0.819672131147541
C:\Users\asus\AppData\Local\Programs\Python\Python37\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [110]:
imp_feature = pd.DataFrame({'Feature': ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal'], 'Importance': xgb.feature_importances_})
plt.figure(figsize=(10,4))
plt.title("barplot Represent feature importance ")
plt.xlabel("importance ")
plt.ylabel("features")
plt.barh(imp_feature['Feature'],imp_feature['Importance'],color = 'red')
plt.show()


ValueError: arrays must all be same length